In [55]:
from utility.file_manager import FileManager
from constants import FileDirectory, Spend
import requests

In [56]:
file_manager = FileManager()

spend_df = file_manager.load_file(FileDirectory.CLEAN_DATA_PATH, Spend.DATA_KEY)

[2024-08-04 00:00:25 +0100] [INFO] [file_manager.py] Successfully loaded file from /Users/hadid/GitHub/ETL/data/clean_data/spend.xlsx


In [57]:
spend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3386 entries, 0 to 3385
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  3386 non-null   int64  
 1   category_a      3386 non-null   object 
 2   category_b      3386 non-null   object 
 3   outlet          3386 non-null   object 
 4   description     1692 non-null   object 
 5   amount          3386 non-null   float64
 6   date            3386 non-null   object 
 7   period          2038 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 211.8+ KB


In [58]:
# remove all na rows in spend_df
# spend_df = spend_df.dropna()
spend_df = spend_df.dropna().reset_index(drop=True)
spend_df.info()
assert spend_df['transaction_id'].notnull().all(), "Found null transaction_id"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1207 entries, 0 to 1206
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  1207 non-null   int64  
 1   category_a      1207 non-null   object 
 2   category_b      1207 non-null   object 
 3   outlet          1207 non-null   object 
 4   description     1207 non-null   object 
 5   amount          1207 non-null   float64
 6   date            1207 non-null   object 
 7   period          1207 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 75.6+ KB


In [59]:
print(spend_df['transaction_id'].min())  # Should be > 0

1349


In [60]:
spend_json = spend_df.to_dict('records')

In [61]:
data_insert = [{"id": 1, "name": None, "age": 25}, {"id": 2, "name": "Jane Smith", "age": 30}]

print(spend_json[0])
print(data_insert[1])

{'transaction_id': 3384, 'category_a': 'Personal Care', 'category_b': 'Mental Health', 'outlet': 'Bromley Counselling', 'description': 'pkey', 'amount': -60.0, 'date': '2024-07-16', 'period': '2024-07'}
{'id': 2, 'name': 'Jane Smith', 'age': 30}


In [62]:
BASE_URL = "http://localhost:8000"

API_USER = "hw_master"
API_PASSWORD = "NQXMq$zFrtmwJBYcvwbhn%X5&Eaz2Nt"

token_response = requests.post(
    f"{BASE_URL}/get-token",
    data={"username": API_USER, "password": API_PASSWORD},
)

access_token = token_response.json()["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}
print(f"\nAccess token for user {API_USER} received\n")



Access token for user hw_master received



In [63]:
load_response = requests.post(
    f"{BASE_URL}/insert-data",
    headers=headers,
    json={
        "db_name": "data_stream_db",
          'table_name': "spend",
          'data': spend_json
    }
)

print(load_response.json())

{'detail': "Error occurred inserting data into table 'spend'"}


In [ ]:
# table_schema = {"id": "INT PRIMARY KEY", "name": "VARCHAR(50)", "age": "INT"}

# response = requests.post(
#     f"{BASE_URL}/create-table",
#     json={
#         "db_name": Settings.TEST_DB_NAME,
#         "table_name": Settings.TEST_TABLE_NAME,
#         "table_schema": table_schema,
#     },
#     headers=headers,
# )
# response_json = response.json()
# print(f"Response create-table: {response_json}\n")

# # ------------------------------
# # Insert Data
# # ------------------------------

# data_insert = [{"id": 1, "name": "None", "age": 25}, {"id": 2, "name": "Jane Smith", "age": 30}]

# response = requests.post(
#     f"{BASE_URL}/insert-data",
#     json={
#         "db_name": Settings.TEST_DB_NAME,
#         "table_name": Settings.TEST_TABLE_NAME,
#         "data": data_insert,
#     },
#     headers=headers,
# )
# response_json = response.json()
# print(f"Response insert-data: {response_json}\n")
